<a href="https://colab.research.google.com/github/razzlestorm/DS-Unit-2-Applied-Modeling/blob/master/module2/Jerimiah_Willhite_assignment_applied_modeling_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Applied Modeling, Module 2

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your work.

- [ ] Plot the distribution of your target. 
    - Classification problem: Are your classes imbalanced? Then, don't use just accuracy.
    - Regression problem: Is your target skewed? If so, let's discuss in Slack.
- [ ] Continue to clean and explore your data. Make exploratory visualizations.
- [ ] Fit a model. Does it beat your baseline?
- [ ] Try xgboost.
- [ ] Get your model's permutation importances.

You should try to complete an initial model today, because the rest of the week, we're making model interpretation visualizations.


## Reading

Top recommendations in _**bold italic:**_

#### Permutation Importances
- _**[Kaggle / Dan Becker: Machine Learning Explainability](https://www.kaggle.com/dansbecker/permutation-importance)**_
- [Christoph Molnar: Interpretable Machine Learning](https://christophm.github.io/interpretable-ml-book/feature-importance.html)

#### (Default) Feature Importances
  - [Ando Saabas: Selecting good features, Part 3, Random Forests](https://blog.datadive.net/selecting-good-features-part-iii-random-forests/)
  - [Terence Parr, et al: Beware Default Random Forest Importances](https://explained.ai/rf-importance/index.html)

#### Gradient Boosting
  - [A Gentle Introduction to the Gradient Boosting Algorithm for Machine Learning](https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/)
  - _**[A Kaggle Master Explains Gradient Boosting](http://blog.kaggle.com/2017/01/23/a-kaggle-master-explains-gradient-boosting/)**_
  - [_An Introduction to Statistical Learning_](http://www-bcf.usc.edu/~gareth/ISL/ISLR%20Seventh%20Printing.pdf) Chapter 8
  - [Gradient Boosting Explained](http://arogozhnikov.github.io/2016/06/24/gradient_boosting_explained.html)
  - _**[Boosting](https://www.youtube.com/watch?v=GM3CDQfQ4sw) (2.5 minute video)**_

In [0]:
#######Using Lending Group Data#####


import os, sys
in_colab = 'google.colab' in sys.modules

# If you're in Colab...
if in_colab:
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Applied-Modeling.git
    !git pull origin master
    
    # Install required python packages
    !pip install -r requirements.txt
    
    # Change into directory for module
    os.chdir('module3')
    
# Ignore this warning: https://github.com/dmlc/xgboost/issues/4300
# xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='xgboost')

Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 77, done.
remote: Total 77 (delta 0), reused 0 (delta 0), pack-reused 77
Unpacking objects: 100% (77/77), done.
From https://github.com/LambdaSchool/DS-Unit-2-Applied-Modeling
 * branch            master     -> FETCH_HEAD
 * [new branch]      master     -> origin/master
Checking out files: 100% (26/26), done.
     |████████████████████████████████| 92kB 4.2MB/s 
     |████████████████████████████████| 112kB 11.1MB/s 
     |████████████████████████████████| 133kB 47.4MB/s 
     |████████████████████████████████| 57.7MB 33.9MB/s 
     |████████████████████████████████| 245kB 47.2MB/s 
     |████████████████████████████████| 614kB 51.1MB/s 
     |████████████████████████████████| 235kB 47.5MB/s 
     |████████████████████████████████| 307kB 48.3MB/s 
     |████████████████████████████████| 215kB 45.7MB/s 
     |████████████████████████████████| 51kB 21.9MB/s 
     |████████████████████████████████| 737kB 41.1M

In [0]:
import pandas as pd

# Stratified sample, 10% of expired Lending Club loans, grades A-D
# Source: https://www.lendingclub.com/info/download-data.action
history_location = '../data/lending-club/lending-club-subset.csv'
history = pd.read_csv(history_location)
history['issue_d'] = pd.to_datetime(history['issue_d'], infer_datetime_format=True)

# Just use 36 month loans
history = history[history.term==' 36 months']

# Index & sort by issue date
history = history.set_index('issue_d').sort_index()

# Clean data, engineer feature, & select subset of features
history = history.rename(columns=                     
    {'annual_inc': 'Annual Income', 
     'fico_range_high': 'Credit Score', 
     'funded_amnt': 'Loan Amount', 
     'title': 'Loan Purpose'})

history['Interest Rate'] = history['int_rate'].str.strip('%').astype(float)
history['Monthly Debts'] = history['Annual Income'] / 12 * history['dti'] / 100

columns = ['Annual Income', 
           'Credit Score', 
           'Loan Amount', 
           'Loan Purpose', 
           'Monthly Debts', 
           'Interest Rate']

history = history[columns]
history = history.dropna()

# Test on the last 10,000 loans,
# Validate on the 10,000 before that,
# Train on the rest
test = history[-10000:]
val = history[-20000:-10000]
train = history[:-20000]

In [0]:
# Assign to X, y
target = 'Interest Rate'
features = history.columns.drop('Interest Rate')

X_train = train[features]
y_train = train[target]

X_val = val[features]
y_val = val[target]

X_test = test[features]
y_test = test[target]

In [0]:
#    Plot the distribution of your target.
import plotly.express as px
fig = px.histogram(history, x="Annual Income", y="Interest Rate")
fig.show()

In [0]:
history.head()

,Annual Income,Credit Score,Loan Amount,Loan Purpose,Monthly Debts,Interest Rate
issue_d,,,,,,
2007-07-01,150000.0,779.0,5000.0,New kitchen,0.000000,7.43
2007-07-01,200000.0,679.0,3000.0,Roofdeck for my new NYC apartment,0.000000,10.28
2007-07-01,95000.0,774.0,5000.0,Mortgage Pre-Payment,303.208333,7.43
2007-07-01,20000.0,684.0,3500.0,Renting Apartment: Need security deposit,25.000000,10.28
2007-08-01,61800.0,749.0,4000.0,Apartment,178.190000,7.75


In [0]:

 #   Fit a model. Does it beat your baseline?

history['Interest Rate'].value_counts(normalize=True)
#Regression problem. Not super skewed, but a little bit.

10.99    0.032776
5.32     0.031960
11.99    0.027925
7.89     0.023330
9.17     0.022612
13.99    0.020919
12.99    0.020516
11.49    0.017269
9.99     0.016354
16.99    0.015124
15.61    0.013923
8.18     0.013500
12.29    0.013215
11.53    0.013156
12.69    0.012516
10.49    0.011473
8.39     0.011345
14.99    0.011296
6.49     0.011021
8.90     0.010922
13.33    0.010863
6.03     0.010834
13.49    0.010735
15.31    0.010204
13.67    0.010145
16.29    0.010125
7.90     0.009761
11.44    0.009545
14.65    0.009308
11.47    0.008718
           ...   
22.50    0.000020
28.80    0.000020
13.87    0.000010
12.41    0.000010
9.70     0.000010
11.41    0.000010
13.04    0.000010
9.51     0.000010
9.64     0.000010
11.59    0.000010
13.08    0.000010
12.36    0.000010
11.72    0.000010
12.22    0.000010
13.36    0.000010
25.00    0.000010
13.62    0.000010
10.01    0.000010
11.09    0.000010
16.07    0.000010
12.09    0.000010
10.83    0.000010
11.63    0.000010
13.84    0.000010
9.01     0

In [0]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    StandardScaler(), 
    RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
)

# Fit on train, score on val
pipeline.fit(X_train, y_train)
print('Validation Accuracy', pipeline.score(X_val, y_val))

Validation Accuracy 0.21105217719780556


In [0]:
 #   Try xgboost. 
from sklearn.metrics import r2_score
from xgboost import XGBRegressor

gBoost = make_pipeline(
    ce.OrdinalEncoder(),
    XGBRegressor(n_estimators=200, objective='reg:squarederror', n_jobs=-1)
)


gBoost.fit(X_train, y_train)
y_pred = gBoost.predict(X_val)
print('Gradient Boosting R^2', r2_score(y_val, y_pred))

Gradient Boosting R^2 0.27488377264062747


In [0]:
#   Get your model's permutation importances.
transformation = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median')
)

X_train_transformed = transformation.fit_transform(X_train)
X_val_transformed = transformation.fit_transform(X_val)

model = XGBRegressor(n_estimators=200, objective='reg:squarederror', random_state=42, n_jobs=-1)
model.fit(X_train_transformed, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=200,
             n_jobs=-1, nthread=None, objective='reg:squarederror',
             random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [28]:
import eli5
from eli5.sklearn import PermutationImportance


permuter = PermutationImportance(
    model,
    scoring='r2',
    n_iter=2,
    random_state=42
)

permuter.fit(X_val_transformed, y_val)
feature_names = X_val.columns.tolist()

eli5.show_weights(
    permuter,
    top=None, 
    feature_names = feature_names
)


Weight,Feature
0.4450 ± 0.0154,Credit Score
0.1922 ± 0.0093,Annual Income
0.0500 ± 0.0020,Loan Purpose
0.0472 ± 0.0102,Monthly Debts
0.0463 ± 0.0008,Loan Amount
